# Decoder Layer

> Implement Transformer's Decoder from scratch

In [ ]:
#| default_exp transformer.decoder

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
from foundation.transformer.efficient_attention import MultiHeadAttention
from foundation.transformer.encoder import PostionWiseFeedForward
from foundation.transformer.embedding import TextEmbedding
from foundation.transformer.positional_encoding import PositionalEncoding
from foundation.transformer.encoder import ResidualLayerNorm

/Users/education/miniforge3/envs/sb-from-scratch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Mask

In [ ]:
#| export
def create_mask(size):
    mask = torch.ones((1, size, size)).triu(1)

In [ ]:
torch.ones((1, 4, 4)).triu(0)

tensor([[[1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 0., 1.]]])

### Decoder Layer

So, query comes from the current decoder layer. key, value come from the last encoder layer

- `x`: the current input for the decoder
    + already through embedding and positional encoding
- `encoder_outputs`:
- `trg_mask`: target mask for all illegal positions
- `src_mask`

In [ ]:
#| export
class DecoderLayer(nn.ModuleList):
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout:float=0.3):
        super().__init__()
        self.norm_1 = ResidualLayerNorm(d_model)
        self.norm_2 = ResidualLayerNorm(d_model)
        self.norm_3 = ResidualLayerNorm(d_model)
        
        self.masked_mha = MultiHeadAttention(d_model, n_heads)
        self.encoder_decoder_mha = MultiHeadAttention(d_model, n_heads)
        self.feed_forward = PostionWiseFeedForward(d_model, d_ff)
    
    def forward(
        self,
        x: torch.Tensor, encoder_output: torch.Tensor,
        trg_mask: torch.Tensor, src_mask: torch.Tensor
    ):
        """_summary_

        Args:
            x (torch.Tensor): the input for the decoder
            encoder_output (_type_): the ouput of the encoder stack
            trg_mask (_type_): the mask for target sequence
            src_mask (_type_): the mask for source sequence
        """
        # shape(x) = [batch_size x trg_seq_len x d_model]
        # shape(encoder_output) = [batch_size x src_seq_len x d_model]
        
        
        # shape(masked_mha) = [batch_size x trg_se_len x d_model]
        # shape(masked_mha_attn_weights) 
        # = [batch_size x n_heads x trg_seq_len x trg_seq_len]
        masked_mha, masked_mha_attn_weights = self.masked_mha(x, x, x, mask=trg_mask)
        
        norm_1 = self.norm_1(masked_mha, x)
        
        # shape(mha) = [batch_size x trg_seq_len x d_model]
        # shape(mha_attn_weights) = [batch_size x n_heads x trg_seq_len x trg_seq_len]
        encoder_decoder_mha, encoder_decoder_mha_attn_weights = self.encoder_decoder_mha(
            pre_q=norm_1, pre_k=encoder_output, pre_v=encoder_output,
            mask=src_mask
        )
        
        norm_2 = self.norm_2(encoder_decoder_mha, norm_1)
        
        feed_forward = self.feed_forward(norm_2)
        
        norm_3 = self.norm_3(feed_forward, norm_2)
        
        return norm_3, masked_mha_attn_weights, encoder_decoder_mha_attn_weights

### Decoder

- `x` is the raw tokenized input
- `encoder_output`: the ouput of encoder
- `trg_mask`
- `srg_mask`

In [ ]:
#| export
class Decoder(nn.Module):
    def __init__(
        self, d_model: int,
        n_heads: int, n_layers: int, d_ff: int,
        dropout: float=0.3
    ):
        super().__init__()
        self.embedding = TextEmbedding(vocab_size=1000, d_model=d_model, padding_idx=0)
        self.positional_encoding = PositionalEncoding(d_model)
        self.dropout = nn.Dropout(dropout)
        self.decoders = nn.ModuleList([
            DecoderLayer(
                d_model,
                n_heads,
                d_ff,
                dropout
            ) for layer in range(n_layers)
        ])
    
    def forward(self, x: torch.Tensor, encoder_output: torch.Tensor, trg_mask: torch.Tensor, src_mask: torch.Tensor):
        # shape(x) = [batch_size x trg_seq_len], raw tokenizer input
        
        # shape(embeddings) = [batch_size x trg_seq_len x d_model]
        embeddings = self.embedding(x)
        # shape(encoding) = [batch_size x trg_seq_len x d_model]
        encoding = self.positional_encoding(embeddings)
        
        for decoder in self.decoders:
            # shape(encoding) = [batch_size x trg_seq_len x d_model]
            # shape(masked_mha_attn_weights) = [batch_size x num_heads x trg_seq_len x trg_seq_len]
            # shape(mha_attn_weights) = [batch_size x num_heads x trg_seq_len x src_seq_len]
            encoding, masked_mha_attn_weights, mha_attn_weights = decoder(
                encoding, encoder_output,
                trg_mask, src_mask
            )
        
        return encoding, masked_mha_attn_weights, mha_attn_weights